### Global variables and methods

In [0]:
# Cargamos datos
traj = pd.read_csv(files_path + traj_path, sep=",")
#gt = pd.read_csv(files_path + gt_path, sep=",")
pred_model = load_model(files_path+model_path+'metar_model.h5')
metar_1_clean = pd.read_csv(files_path+metar1_path, sep=",") # 30 minutos antes de aterrizaje
metar_2_clean = pd.read_csv(files_path+metar2_path, sep=",") # Momento de aterrizaje

In [0]:
# Normalizamos los datos de trayectoria respecto de los máximos y mínimos (no siguen una distribución gaussiana)
def normalize_data(x_data, min_value, max_value):
  # Normalizamos datos de entrada entre un mínimo y máximo para obtener un valor entre 0 y 1
  # x_data tiene shape = (samples, timestemps)
  x_norm = np.zeros(x_data.shape)
  x_norm = (x_data-min_value)/(max_value-min_value)

  return x_norm

# Denormalizar los datos para crear la predicción en valores (coordenadas y altitud) reales
def denormalize_data(norm_data, min_value, max_value):
  # Desormalizamos datos normalizados entre 0 y 1 para obtener entre mínimo y máximo 
  # x_data tiene shape = (samples, timestemps)
  x_data = np.zeros(norm_data.shape)
  x_data = (norm_data*(max_value-min_value))+min_value

  return x_data

# Estandarizamos los datos de metar respecto de la media y std de los datos de entrenamiento
def standarize_data(x_data):
  # Se utiliza la media y std de los datos de entrenamiento para estandarizar lso datos de test

  x_standarized = np.zeros(x_data.shape)

  for i in range(len(x_data.T)):
    std = np.std(x_data.T[i])
    if std ==0: std=1 # Si la std es 0, cambiamos por 1 para evitar dividir entre 0
    x_standarized.T[i] = (x_data.T[i]-np.mean(x_data.T[i]))/std
    
  return x_standarized

## TODO: modificar este método para que sea funcional en este caso y poder crear las particiones de test y train

# Mezclar datos de train y test y crear particiones
def shuffle_and_partition(x_traj, id_list, y, trainP):
  # x=inputs, y=outputs, trainP=porcentaje de train
  s = np.arange(x_traj.shape[0])
  np.random.shuffle(s)
  x_traj, id_list, y = x_traj[s], np.array(id_list)[s], y[s]

  # Asignamos particiones de train y test
  n_part = round(trainP*len(x_traj))

  x_traj_train, x_traj_test, id_list_train, id_list_test, y_train, y_test = x_traj[:n_part], x_traj[n_part:], id_list[:n_part], id_list[n_part:], y[:n_part], y[n_part:]
  return x_traj_train, x_traj_test, id_list_train, id_list_test, y_train, y_test

### Tratamiento datos de trayectoria

Pasamos las variables a formato float y nos quedamos los datos

In [0]:
import re
variables = ['hour', 'minute', 'second', 'altitude', 'direction', 'latitude', 'longitude', 'speed']
for v in variables:
  n = 0
  for i in traj[v]:  
    result = re.search('\[(.*)\]', i)
    traj[v][n] = [float(j) for j in result.group(1).split(',')]
    n +=1
fl_id_list = list(traj['fl_id'])
threshold = list(traj['threshold'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [0]:
# Separar una serie de datos secuenciales de una sola variable en muestras diferentes y la salida - Preparación para LSTM univariate
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		end_ix = i + n_steps # Fin de secuencia
		if end_ix > len(sequence)-1: # Comprobar si hemos rebasado el final
			break
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix] # Separar entre entrada y salida
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)


In [0]:
import numpy as np
# Separar una serie de datos secuenciales de una varas variables en muestras diferentes y la salida - Preparación para LSTM multivariate
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

Definición del modelo

In [0]:
# define model
model = Sequential()
#model.add(LSTM(32,dropout = 0.1,recurrent_dropout=0.5,activation='relu',return_sequences=True, input_shape=(n_steps, n_features)))
#model.add(layers.Embedding(n_features, 512))
model.add(LSTM(30,dropout = 0.2,recurrent_dropout = 0.2 ,return_sequences = True))
model.add(LSTM(60,dropout = 0.2,recurrent_dropout = 0.2))
#model.add(Dense(64,activation = 'relu'))
#model.add(Dropout(0.2))
model.add(Dense(n_features))
RMS = optimizers.RMSprop()
#Adam = optimizers.Adam()
model.compile(optimizer=RMS, loss='mse')
#model.compile(optimizer='Adam', loss='mae')
# fit model
model.fit(X, Y, epochs=epochs, batch_size=batch_size, verbose=1,validation_split = validation_split)

#Save model
model.save(files_path+model_path+'uv_traj_model.h5')


Train on 33310 samples, validate on 8328 samples
Epoch 1/25
33310/33310 [==============================] - 26s 768us/step - loss: 0.0138 - val_loss: 0.0036
Epoch 2/25
33310/33310 [==============================] - 12s 354us/step - loss: 0.0066 - val_loss: 0.0030
Epoch 3/25
33310/33310 [==============================] - 12s 356us/step - loss: 0.0058 - val_loss: 0.0028
Epoch 4/25
33310/33310 [==============================] - 12s 356us/step - loss: 0.0054 - val_loss: 0.0032
Epoch 5/25
33310/33310 [==============================] - 12s 350us/step - loss: 0.0051 - val_loss: 0.0023
Epoch 6/25
33310/33310 [==============================] - 12s 350us/step - loss: 0.0052 - val_loss: 0.0030
Epoch 7/25
33310/33310 [==============================] - 12s 356us/step - loss: 0.0051 - val_loss: 0.0023
Epoch 8/25
33310/33310 [==============================] - 12s 352us/step - loss: 0.0050 - val_loss: 0.0026
Epoch 9/25
33310/33310 [==============================] - 12s 351us/step - loss: 0.0050 - val_l